# **BASE DE DADOS**

- Instalação e Importações

  Execute esta células abaixo primeiro para garantir que as bibliotecas estejam presentes.

In [2]:
import heapq
import networkx as nx
import wikipedia

In [5]:
# Configurando para a Wikipédia em Português (essencial para seus seeds)
wikipedia.set_lang("pt")

- Configuração dos Seeds e Stopwords

  Aqui definimos os 5 temas e as palavras de parada para evitar páginas administrativas (como ISBN, Categorias, etc.).

In [1]:
# Definição dos SEEDS (Pontos de partida)
SEEDS = [
    "Transformer (modelo de aprendizado de máquina)", # Ajustado para o título correto na PT-Wiki
    "The Beatles",
    "Revolução Francesa",
    "One Piece",
    "Rio Grande do Norte"
]

# Lista de Stopwords
STOPS = (
    "International Standard Serial Number",
    "International Standard Book Number",
    "National Diet Library",
    "International Standard Name Identifier",
    "Digital Object Identifier",
    "Arxiv",
    "PubMed",
    "Bibcode",
    "Jstor",
    "Doi (Identificador)",
    "Isbn (Identificador)",
    "Pmid (Identificador)",
    "Arxiv (Identificador)",
    # Adições específicas para limpar a Wikipédia em PT
    "Wikipédia",
    "Ajuda",
    "Ficheiro",
    "Categoria",
    "Portal",
    "Especial",
    "Livro",
    "Predefinição"
)

print(f"Seeds configurados: {SEEDS}")

Seeds configurados: ['Transformer (modelo de aprendizado de máquina)', 'The Beatles', 'Revolução Francesa', 'One Piece', 'Rio Grande do Norte']


- Construção da Rede
  
  Este bloco constitui o núcleo da coleta de dados. A implementação clássica do algoritmo de Busca em Largura (Breadth-First Search - BFS) foi adaptada para suportar múltiplos seeds e atingir a profundidade alvo de nível 2 (altura < 3).

  Conforme destacado no Requisito 4, a expansão da rede até este nível acarreta um crescimento exponencial no número de nós e arestas, gerando uma demanda computacional proibitiva para uma busca exaustiva simples. Para viabilizar a coleta e manter o foco nos temas de interesse, substituímos a abordagem padrão por uma estratégia de Busca Gulosa (Greedy Search) com Heurística.

### Metodologia Aplicada:

1.  **Estrutura de Dados (Priority Queue):**
    Substituímos a estrutura de fila convencional (FIFO), típica do BFS, por uma Fila de Prioridade implementada sobre um Min-Heap (heapq). Essa alteração permite que o algoritmo abandone o processamento sequencial cego em favor de uma abordagem ordenada pelo "custo", onde nós de maior relevância (menor score) são processados primeiro.

2.  **Heurística de Relevância (Score):**
    Implementamos a função calcular_score_turbo que processa os SEEDS de entrada em tempo de execução, aplicando técnicas simplificadas de PLN (Processamento de Linguagem Natural):
   * **Extração de Radicais (Stemming):** O algoritmo "limpa" os Seeds e reduz palavras longas aos seus radicais (ex: "Transformer" torna-se "trans"). Isso permite capturar variações (como "Transformação" ou "Transformers"), recuperando o volume de conexões sem perder o contexto.
    * **Bonificação (-50 pontos):** Se o título da página contiver palavras-chave relacionadas aos 5 SEEDs originais (ex: "Beatles", "Revolução", "Brasil"). Isso mantém a coesão temática da rede.
    * **Penalidade (+50 pontos):** Se o título for excessivamente longo ou identificado como índices ("Lista de...", "List"), que possuem baixo valor topológico.
    * **Poda (Pruning) por Página:** Para evitar a explosão combinatória e o ruído, aplicamos um corte rígido (MAX_FILHOS_POR_PAGINA). Após calcular o score de todos os links de uma página, o algoritmo ordena os candidatos e retém apenas os Top N (ex: 20) melhores. Isso garante que apenas a "nata" das conexões seja adicionada ao grafo final.

Dessa forma, garantimos a construção de uma base de dados densa, tematicamente coesa e que respeita os limites computacionais do projeto.

In [8]:
def gerar_keywords_inteligentes(lista_seeds):
    keywords = set()
    for seed in lista_seeds:
        # 1. Limpa parênteses e converte para minúsculo
        limpo = seed.split("(")[0].lower()
        palavras = limpo.split()
        
        for p in palavras:
            # 2. Aceita palavras a partir de 3 letras (Salva 'Rio', 'One', 'Usa')
            if len(p) >= 3:
                # 3. TRUQUE DO VOLUME: Se a palavra for longa (>5), pega só o radical
                # Ex: "Revolução" vira "revol". "Transformer" vira "trans"
                if len(p) > 5:
                    keywords.add(p[:5]) 
                else:
                    keywords.add(p)
    
    # Remove palavras muito comuns que podem ter passado (ruído)
    palavras_banidas = {"das", "dos", "the", "for", "com", "sem", "sob"}
    keywords = keywords - palavras_banidas
    
    print(f"Keywords Geradas (Raízes): {keywords}")
    return keywords

GLOBAL_KEYWORDS = gerar_keywords_inteligentes(SEEDS)

Keywords Geradas (Raízes): {'franc', 'piece', 'trans', 'grand', 'beatl', 'one', 'norte', 'revol', 'rio'}


In [7]:
MAX_FILHOS_POR_PAGINA = 20

def calcular_score_turbo(titulo_link, keywords_set):
    """
    Menor é melhor.
    Score Base: 100
    - Contém radical do Seed? -50
    - Título gigante ou Lista? +50
    """
    score = 100
    titulo_lower = titulo_link.lower()

    # Verifica se ALGUMA keyword está contida no título
    # Ex: se keyword é "revol", vai dar match em "Revolução" e "Revolta"
    for k in keywords_set:
        if k in titulo_lower:
            score -= 50
            break 

    # Penalidades
    listas = {"lista", "list", "índice"}
    if len(titulo_link) > 60 or any(l in titulo_lower for l in listas):
        score += 50

    return score

# --- INICIALIZAÇÃO ---
priority_queue = []
for seed in SEEDS:
    heapq.heappush(priority_queue, (0, 0, seed))

todo_set = set(SEEDS)
done_set = set()
g = nx.DiGraph()

print(f"Iniciando Coleta Otimizada (Máx {MAX_FILHOS_POR_PAGINA} filhos/página)...")

# --- LOOP PRINCIPAL ---
while priority_queue:
    layer, score, page = heapq.heappop(priority_queue)

    if layer >= 3: # Profundidade máxima
        continue

    if page in done_set:
        continue

    done_set.add(page)
    print(f"Camada {layer} | Processando: {page} | Grafo: {len(g)} nós")

    try:
        wiki = wikipedia.page(page, auto_suggest=False)
        links_brutos = wiki.links
    except Exception:
        continue

    candidatos = []
    for link in links_brutos:
        link_str = link # wikipedia.links já retorna strings, não precisa .title() as vezes
        if link_str not in STOPS and not link_str.startswith("Lista De"):
            # Passamos o GLOBAL_KEYWORDS gerado lá fora
            s = calcular_score_turbo(link_str, GLOBAL_KEYWORDS)
            candidatos.append((s, link_str))

    # Ordena e Corta
    candidatos.sort(key=lambda x: x[0])
    melhores_filhos = candidatos[:MAX_FILHOS_POR_PAGINA]

    for s, link_escolhido in melhores_filhos:
        g.add_edge(page, link_escolhido)

        if link_escolhido not in todo_set and link_escolhido not in done_set:
            heapq.heappush(priority_queue, (layer + 1, s, link_escolhido))
            todo_set.add(link_escolhido)

print("-" * 30)
print(f"COLETA FINALIZADA!")
print(f"Total: {len(g)} nós e {nx.number_of_edges(g)} arestas.")

Iniciando Coleta Otimizada (Máx 20 filhos/página)...
Camada 0 | Processando: One Piece | Grafo: 0 nós
Camada 0 | Processando: Revolução Francesa | Grafo: 21 nós
Camada 0 | Processando: Rio Grande do Norte | Grafo: 21 nós
Camada 0 | Processando: The Beatles | Grafo: 42 nós
Camada 0 | Processando: Transformer (modelo de aprendizado de máquina) | Grafo: 63 nós
Camada 1 | Processando: ...Baby One More Time | Grafo: 63 nós
Camada 1 | Processando: 1 (Beatles album) | Grafo: 82 nós
Camada 1 | Processando: 20 Greatest Hits (Beatles album) | Grafo: 93 nós
Camada 1 | Processando: 2025 Indonesian protests | Grafo: 110 nós
Camada 1 | Processando: 34 Montagu Square, Marylebone | Grafo: 130 nós
Camada 1 | Processando: 4 by the Beatles | Grafo: 143 nós
Camada 1 | Processando: A Collection of Beatles Oldies | Grafo: 150 nós
Camada 1 | Processando: A Salute to the Beatles: Once upon a Time | Grafo: 155 nós
Camada 1 | Processando: A Taste of Honey (band) | Grafo: 166 nós
Camada 1 | Processando: A Taste 

c:\Users\Alice\AppData\Local\pypoetry\Cache\virtualenvs\analise-redes-wiki-UZXaxLaG-py3.14\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Alice\AppData\Local\pypoetry\Cache\virtualenvs\analise-redes-wiki-UZXaxLaG-py3.14\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Camada 1 | Processando: Beatles (novel) | Grafo: 269 nós
Camada 1 | Processando: Beatles-Platz | Grafo: 288 nós
Camada 1 | Processando: Bom Jesus, Rio Grande do Norte | Grafo: 288 nós
Camada 1 | Processando: Brejinho, Rio Grande do Norte | Grafo: 292 nós
Camada 1 | Processando: Brook (One Piece) | Grafo: 297 nós
Camada 1 | Processando: Butsu Zone | Grafo: 303 nós
Camada 1 | Processando: Caiçara do Norte | Grafo: 322 nós
Camada 1 | Processando: Caiçara do Rio do Vento | Grafo: 325 nós
Camada 1 | Processando: Campo Grande, Rio Grande do Norte | Grafo: 329 nós
Camada 1 | Processando: Campo Redondo, Rio Grande do Norte | Grafo: 333 nós
Camada 1 | Processando: Captaincy of Rio Grande | Grafo: 336 nós
Camada 1 | Processando: Caraúbas, Rio Grande do Norte | Grafo: 354 nós
Camada 1 | Processando: Cerro Corá, Rio Grande do Norte | Grafo: 355 nós
Camada 1 | Processando: Coronel Ezequiel | Grafo: 355 nós
Camada 1 | Processando: Coronel João Pessoa | Grafo: 358 nós
Camada 1 | Processando: Encanto,

- Tratamento de Dados (Limpeza)
  Remoção de duplicatas (ex: plurais) e auto-loops, conforme feito no notebook do professor.

In [18]:
print(f"Iniciando limpeza em grafo de {len(g)} nós...")
print("Isso pode demorar um pouco devido ao tamanho da rede.")

# 1. Remover auto-loops (Rápido e seguro)
g.remove_edges_from(nx.selfloop_edges(g))

# 2. Fundir plurais (Ex: 'Palavra' e 'Palavras') com verificação
duplicates = [(node, node + "s") for node in g if node + "s" in g]
for u, v in duplicates:
    if u in g and v in g: # <--- PROTEÇÃO: Só funde se ambos ainda existirem
        g = nx.contracted_nodes(g, u, v, self_loops=False)

# 3. Fundir hífens (Ex: 'One-Punch Man' e 'One Punch Man') com verificação
duplicates_hyphen = [(node, node.replace("-", " ")) for node in g
                     if node.replace("-", " ") in g and node != node.replace("-", " ")]

for u, v in duplicates_hyphen:
    if u in g and v in g: # <--- PROTEÇÃO: Só funde se ambos ainda existirem
        g = nx.contracted_nodes(g, u, v, self_loops=False)

# Limpeza técnica de atributos (para evitar erros no Gephi)
for node in g.nodes():
    g.nodes[node].pop('contraction', None)
for u, v in g.edges():
    g.edges[u, v].pop('contraction', None)

print(f"Limpeza concluída! Grafo atual: {len(g)} nós.")

Iniciando limpeza em grafo de 3425 nós...
Isso pode demorar um pouco devido ao tamanho da rede.
Limpeza concluída! Grafo atual: 3425 nós.


- Filtragem e Exportação
  
  Gera o arquivo final .graphml.

In [20]:
# Filtra o "Core" da rede: Mantém apenas nós com grau >= 2
# (Nós que têm pelo menos 2 conexões de entrada ou saída)
core = [node for node, deg in dict(g.degree()).items() if deg >= 2]
gsub = g.subgraph(core).copy()

print("-" * 30)
print(f"Grafo Original: {len(g)} nós")
print(f"Grafo Final (Core): {len(gsub)} nós")
print("-" * 30)

# Salva o arquivo para o Gephi
nome_arquivo = "trabalho_final_validacao.graphml"
nx.write_graphml(gsub, nome_arquivo)

print(f"Arquivo '{nome_arquivo}' salvo com sucesso!")
print("Faça o download na aba de arquivos (ícone da pasta) à esquerda.")

------------------------------
Grafo Original: 3425 nós
Grafo Final (Core): 978 nós
------------------------------
Arquivo 'trabalho_final_validacao.graphml' salvo com sucesso!
Faça o download na aba de arquivos (ícone da pasta) à esquerda.
